This file contains the code for solving the quaternion $\ell$ isogeny problem by navigating isogeny volcanoes using solutions to the quaternion embedding problem. This has only been tested in the case $p\equiv 3 \mod 4$.

In [124]:
load("utils.sage")

def quaternion_ell_ideal_path(O_start, O_target, ell, m=None, randomize_qep=False, debug=False):
    """
    Solves the ell isogeny path problem from certain curves, for small prime characteristic (i.e. < 2^50).
    `O_start` - Starting maximal order, where QEP is solved for large disc orders, so it must have small denominators.
                E.g. O1728 when p = 3 mod 4 is an optimal choice
    `O_target` - The maximal order to walk to
    `ell` - The prime to find an ell^n ideal
    `randomize_qep` - Should we use the randomized algorithm to solve the quaternion embedding problem. Works better for larger primes $p$ with fast factorization is unrealistic.
    `m` - Set to give an upper bound on path length as ell^m.
            Default is roughly  m ~ p^(1/6)
            Increasing can reduce number of iterations, but take longer to solve quadratic forms in each iteration.
            Decreasing too much can make it impossible to solve, or take very long time.
    """
    QuatAlg = O_target.quaternion_algebra()
    Zell = Integers(ell)
    
    ### STEP 1: Do short ell**... walk from target order until find order with "nice" quadratic embedding

    for STEP1_I in enum_cyclic_l_ideals(O_target, ell):
        O_target_new = STEP1_I.right_order()
        # Get quadratic order from smallest element
        smallest_elt = O_target_new.unit_ideal().reduced_basis()[1]
        d = smallest_elt.reduced_norm()
        t = smallest_elt.reduced_trace()
        Ofrak, g = quadratic_order_from_norm_trace(d, t, generator=True)
        # Check ell split in O
        K_disc = Ofrak.number_field().disc()
        if ell != 2:
            if mod(K_disc, ell) == 0: continue
            if not Zell(K_disc).is_square(): continue
        else:
            if mod(K_disc, 8) != 1: continue
        if debug:
            print("Found starting order with ell split")

        ### STEP 2: Walk to top of the oriented isogeny volcano with respect to the "nice" quadratic embedding, giving Oright

        # Solve QEP on O_start to find embedding 
        n = -1
        while True: # For random case could make a good estimate for n, change to look for primitive embeddings, and remove loop
            n += 1
            suborder_t = (ell**n) * t
            suborder_d = (ell**(2*n)) * d
            elt = None
            if randomize_qep:
                sln = find_element_defining_embedding_randomized(O_start, suborder_d, suborder_t, filter_func=None, debug=False)
                if len(sln) == 1:
                    elt = sln
            else:
                elt = find_element_defining_embedding(O_start, suborder_d, suborder_t)
            if elt != None:
                break
        # Walk to top
        if debug:
            print("Walked " + str(n) + " steps to rim")
        STEP2_I = O_start*elt + O_start*(ell**n)
        Oright = STEP2_I.right_order()
        Oright_origin = Oright

        ### STEP 3: Get shortest connecting ideal from Oright to O. Step along any ell**... factor of the ideal. Update Oright to be the right order of this new ideal

        # Get remaining ideal
        Ia, gamma2 = small_equivalent_ideal(connecting_ideal(Oright, O_target_new), True)
        N = Ia.norm()
        # Have to remove norm ell parts of this
        exp_removed = 0
        STEP3_I = Oright.unit_ideal()
        while mod(N, ell) == 0:
            N = N / ell
            exp_removed += 1
        if exp_removed > 0:
            STEP3_I = Ia + Oright*(ell**exp_removed)
            Oright = STEP3_I.right_order()
            Ia = small_equivalent_ideal(connecting_ideal(Oright, O_target_new)) # Could do something cleverer here, but its good enough
        if N == 1:
            if debug:
                print("FINISHED EARLY")
                print("")
            complete_ideal = STEP2_I * STEP3_I * (gamma2.inverse() * STEP1_I.conjugate() * gamma2)
            # Make solution cyclic
            complete_ideal = make_ell_cyclic(complete_ideal, ell)
            return complete_ideal

        ### STEP 4: For remaining ideal from Oright to O, find corresponding quadratic ideal, find equivalent horizontal norm ell**... ideal, and convert it back to quaternion ideal

        # Quaternion Ideal to Quadratic Ideal
        elt1 = elt / (ell**n)
        bs = QuaternionLattice(Ia).intersection(QuaternionLattice([QuatAlg(1), elt1])).upper_hnf_basis()
        gen1 = bs[0]
        gen2 = bs[1]
        #if Ia != Oright*gen1 + Oright*gen2:
        #    print("Issue getting quadratic ideal. Restarting.")
        #    break
        k1 = ZZ(gen2[1] / elt1[1])
        k2 = ZZ(gen2 - elt1*k1)
        k3 = ZZ(gen1)
        fraka = Ofrak.ideal([k3, k2 + k1*g]) # This is the quadratic ideal
        # Getting norm form of quadratic ideal
        R.<x0,x1> = PolynomialRing(QQ)
        nx = (x0*k3 + x1*(k2 + k1*g))
        nxbar = (x0*k3 + x1*(k2 + k1*g.conjugate()))
        fraka_norm_form = QuadraticForm(nx * nxbar)
        a, b, c = fraka_norm_form.coefficients()
        fraka_norm_form_bin = BinaryQF(a,b,c)
        # Attempt to solve norm form finding N ell**m solution - by guessing an m for which there should be a solution
        #    Previously used a loop over m here, but it seems slower
        #    Size of m should be roughly sqrt of size of disc of order, as that is expected class group size. And maybe 4th root as that is expected size of order of I
        if m == None:
            m = ceil((Ofrak.discriminant()**(1/4)).abs())
        sln = fraka_norm_form_bin.solve_integer(N*(ell**m))
        if sln != None:
            # From solution find equivalent quadratic ideal of norm ell**m
            x0, x1 = sln
            beta = x0*k3 + x1*(k2 + k1*g)
            betaconj = beta.conjugate()
            # The norm ell**m quadratic ideal is then:   frakJ = Ofrak.ideal([betaconj, ell**m])
            # Map quadratic ideal generator betaconj to quaternion, by first writing  betaconj = x1 + x2*g , then quaternion is  x1 + x2*elt1
            x2 = ZZ(betaconj[1] / g[1])
            x1 = ZZ(betaconj - x2*g)
            alpha = QuatAlg(x1 + x2*elt1)
            STEP4_I = Oright*alpha + Oright*(ell**m)    # Norm  ell**m  quaternion ideal
            complete_ideal = STEP2_I * STEP3_I * STEP4_I * ((gamma2*alpha/N).inverse() * STEP1_I.conjugate() * (gamma2*alpha/N))
            # Make solution cyclic
            complete_ideal = make_ell_cyclic(complete_ideal, ell)
            return complete_ideal
        if debug:
            print("No solutions found.")
            print("")

In [125]:
norms = []

In [171]:
p = (2**30).next_prime()
assert(mod(p, 4) == 3)

QuatAlg = QuaternionAlgebra(p)
O_start = QuatAlg.maximal_order()
O_target = random_maximal_order(QuatAlg)

ell = 3

I = quaternion_ell_ideal_path(O_start, O_target, ell, m=None, debug=True)

# Check left and right orders correct
assert(I.left_order() == O_start)
assert(check_orders_conjugate(I.right_order(), O_target))
# Check ideal norm is power of ell
assert(is_power_of_ell(I.norm(), ell))
# check ideal is integral
for b in I.basis():
    assert(b in O_start)
# check ideal is cyclic
assert(is_cyclic(I))

norms.append(RR(log(I.norm(), p)))

Found starting order with ell split
Walked 12 steps to rim
No solutions found.

Found starting order with ell split
Walked 14 steps to rim


<string>:325: DeprecationWarning: is_equivalent is deprecated, please use is_left_equivalent or is_right_equivalent accordingly instead
See https://github.com/sagemath/sage/issues/37100 for details.


After running this several times for $30$-bit primes we get the following array.
Each entry $e$ corresponds to an execution, and a resulting ideal of norm $\ell^{*} = p^e$. For the average case the ideal seems to be of norm $\sim p^{1.47}$ (where we take the average of the exponents, rather than norms, to give a more accurate meaning of 'size')

In [177]:
norms

[1.16230583370601,
 2.64160416751366,
 1.47929833380765,
 1.74345875055902,
 2.11328333401093,
 1.05664166700547,
 1.10947375035574,
 2.74726833421421,
 0.739649166903826,
 0.845313333604373,
 1.00380958365519,
 1.05664166700547,
 0.633985000203279,
 0.898145416954646,
 1.26797000040656,
 0.950977500304919,
 1.53213041715793,
 2.27177958406175,
 1.74345875055902,
 1.74345875055902,
 1.47929833380765,
 2.11328333401093,
 1.21513791705629,
 1.05664166700547,
 1.74345875055902,
 2.21894750071148,
 1.37363416710711,
 1.69062666720875,
 1.37363416710711,
 1.00380958365519]

In [176]:
sum(norms)/len(norms)

1.46697084769259

We also ran the code $30$ times for a $50$-bit prime ...

In [63]:
norms

[3.04312800138462,
 8.97088775408173,
 18.0685725082212,
 4.91338375223558,
 1.14117300051923,
 16.1349182573413,
 4.27939875194711,
 16.5787077575433,
 19.7803320090000,
 11.7287225053365,
 9.98526375454327,
 1.10947375050481,
 5.10357925232212,
 13.9476700063462,
 10.6509480048462,
 4.02580475183173,
 18.7025575085096,
 20.7630087594471,
 14.9937452568221,
 22.8868585104135,
 1.17287225053365,
 10.8411435049327,
 21.2384975096635,
 7.70291775350481,
 5.26207550239423,
 5.45227100248077,
 2.82123325128365,
 17.8149785081058,
 14.1695647564471,
 13.0917902559567]

In [65]:
sum(norms)/len(norms)

10.8791826049500